# Model Training Pipeline
## Training Steps
1. Load Dataset
2. Data Tokenization
3. Model Parametrization
4. Model Training
5. Model Evaluation

In [1]:
import pandas as pd
import os
from nltk.tokenize import TreebankWordTokenizer
print("Working Directory:", os.getcwd())

Working Directory: c:\Users\arthu\Documents\GitHub\ner-using-bert\BERT_Experiment


### Step One 
# Load Dataset
We are using datasets annotated in [Doccano](https://github.com/doccano/doccano), which outputs a **.jsonl** file with a list of all the documents and their labels.

In [3]:
data_file_path = "acerpi_dataset/train/annotated"
dataset_filename = "labeled_ufrgs_sentences.jsonl"

ufrgs_documents = pd.read_json(os.path.join(data_file_path, dataset_filename), orient='record', lines=True)

In [4]:
#Filter the dataset to remove all sentences without relevant labels besides 'O'
ufrgs_documents = ufrgs_documents[ufrgs_documents['label'].str.len() > 0]

In [5]:
# Keep only the relevant columns: the sentence text, the labels and the id to link back to the original document later on.
ufrgs_labeled_sentences = ufrgs_documents[['text', 'label', 'sentence_id']]

## Step Two
# Data Tokenization
Let's tokenize the sentences and tokens using the TreebankWordTokenizer.  

The first method **tokenize** will split our sentence and return a list of words which we'll organize in a 'tokenized_sentences' list.  

The second method **span_tokenize** will return the first and last indices of the characters in each token in reference to the orignal sentence.  
This allows us to recreate the sentence later on. We'll organize them in a 'token_positions' list.

In [6]:
from transformers import AutoTokenizer
checkpoint = 'neuralmind/bert-large-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)

In [14]:
t = tokenizer(ufrgs_labeled_sentences['text'][4], is_split_into_words=False)
word_ids = t.word_ids()
tokens_by_word_id = {}

for idx, word in enumerate(word_ids):
    if word is not None:
        tokens_by_word_id.setdefault(word, []).append(t.tokens()[idx])

In [15]:
ufrgs_labeled_sentences['text'][4]

'Autorizar o afastamento do país de ANDRE DIAS MORTARI, Assistente em Administração, lotado e em exercício na Escola de Administração, com a finalidade de participar do "IV Congreso Internacional Red Pilares", em Cartagena - Colombia, no período compreendido entre 29/08/2016 e 02/09/2016, com ônus limitado.'

In [16]:
tokens_by_word_id

{0: ['Autor', '##izar'],
 1: ['o'],
 2: ['afastamento'],
 3: ['do'],
 4: ['país'],
 5: ['de'],
 6: ['AN', '##DR', '##E'],
 7: ['D', '##IA', '##S'],
 8: ['M', '##OR', '##TA', '##RI'],
 9: [','],
 10: ['Assis', '##tente'],
 11: ['em'],
 12: ['Administração'],
 13: [','],
 14: ['lo', '##tado'],
 15: ['e'],
 16: ['em'],
 17: ['exercício'],
 18: ['na'],
 19: ['Escola'],
 20: ['de'],
 21: ['Administração'],
 22: [','],
 23: ['com'],
 24: ['a'],
 25: ['finalidade'],
 26: ['de'],
 27: ['participar'],
 28: ['do'],
 29: ['"'],
 30: ['IV'],
 31: ['Con', '##gre', '##so'],
 32: ['Internacional'],
 33: ['Red'],
 34: ['Pil', '##ares'],
 35: ['"'],
 36: [','],
 37: ['em'],
 38: ['Carta', '##gen', '##a'],
 39: ['-'],
 40: ['Colo', '##mbia'],
 41: [','],
 42: ['no'],
 43: ['período'],
 44: ['compree', '##ndido'],
 45: ['entre'],
 46: ['29'],
 47: ['/'],
 48: ['08'],
 49: ['/'],
 50: ['2016'],
 51: ['e'],
 52: ['02'],
 53: ['/'],
 54: ['09'],
 55: ['/'],
 56: ['2016'],
 57: [','],
 58: ['com'],
 59: ['[U

In [17]:
tokens_by_word_id

{0: ['Autor', '##izar'],
 1: ['o'],
 2: ['afastamento'],
 3: ['do'],
 4: ['país'],
 5: ['de'],
 6: ['AN', '##DR', '##E'],
 7: ['D', '##IA', '##S'],
 8: ['M', '##OR', '##TA', '##RI'],
 9: [','],
 10: ['Assis', '##tente'],
 11: ['em'],
 12: ['Administração'],
 13: [','],
 14: ['lo', '##tado'],
 15: ['e'],
 16: ['em'],
 17: ['exercício'],
 18: ['na'],
 19: ['Escola'],
 20: ['de'],
 21: ['Administração'],
 22: [','],
 23: ['com'],
 24: ['a'],
 25: ['finalidade'],
 26: ['de'],
 27: ['participar'],
 28: ['do'],
 29: ['"'],
 30: ['IV'],
 31: ['Con', '##gre', '##so'],
 32: ['Internacional'],
 33: ['Red'],
 34: ['Pil', '##ares'],
 35: ['"'],
 36: [','],
 37: ['em'],
 38: ['Carta', '##gen', '##a'],
 39: ['-'],
 40: ['Colo', '##mbia'],
 41: [','],
 42: ['no'],
 43: ['período'],
 44: ['compree', '##ndido'],
 45: ['entre'],
 46: ['29'],
 47: ['/'],
 48: ['08'],
 49: ['/'],
 50: ['2016'],
 51: ['e'],
 52: ['02'],
 53: ['/'],
 54: ['09'],
 55: ['/'],
 56: ['2016'],
 57: [','],
 58: ['com'],
 59: ['[U

In [ ]:
tokenized_sentences = []
token_positions = []
for idx, sentence in annotated_sentences['text'].items():
    tokenized_sentences.append(TreebankWordTokenizer().tokenize(sentence))
    token_positions.append(list(TreebankWordTokenizer().span_tokenize(sentence)))


print(tokenized_sentences[0])
print(token_positions[0])